Here we evaluate the performance of different models generated.

Lets import the test data first

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ng6mKmGtGPTwjl1w-iDYC0hvQW0CDv_bM2qXoyl3oZRL',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = cos.get_object(Bucket='foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',Key='df_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body)


In [4]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_8e01ca4d30994b239a53c23104c881fd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ng6mKmGtGPTwjl1w-iDYC0hvQW0CDv_bM2qXoyl3oZRL',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_8e01ca4d30994b239a53c23104c881fd.get_object(Bucket='foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',Key='df_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_train = pd.read_csv(body)



In [5]:
import numpy as np
from keras.models import model_from_json
from keras.models import load_model
import json
X_train = df_train[df_train.columns[1:17]] 
Y_train = df_train['Confirmed']
X_test = df_test[df_test.columns[1:17]] 
Y_test = df_test['Confirmed']
X_train_np=X_train.to_numpy()
Y_train_np=Y_train.to_numpy()
X_test_np=X_test.to_numpy()
Y_test_np=Y_test.to_numpy()

Using TensorFlow backend.


In [6]:
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-e8071002-7c9e-408a-9f8f-7095f09bcb51',
    'IBM_API_KEY_ID': 'ng6mKmGtGPTwjl1w-iDYC0hvQW0CDv_bM2qXoyl3oZRL',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',
    'FILE': 'Food_Supply_kcal_Data.csv'
}

Download models from Cloud object store.

In [7]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='model1.pickle',Filename='model1.pickle')
cos.download_file(Bucket=credentials_1['BUCKET'],Key='model2.pickle',Filename='model2.pickle')
cos.download_file(Bucket=credentials_1['BUCKET'],Key='model3.h5',Filename='model3.h5')
cos.download_file(Bucket=credentials_1['BUCKET'],Key='model4.h5',Filename='model4.h5')

First consider the deep learning model without regularization.

In [8]:
model3=load_model("model3.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [9]:
from sklearn.metrics import mean_squared_error
y3_pred=model3.predict(X_test_np)
for i in range(len(y3_pred)):
    if(y3_pred[i]<0):
        y3_pred[i]=0
model_3_mse=mean_squared_error(Y_test_np, y3_pred)
model_3_mse

0.004365087735385307

Now consider the deep learning model with regularization.

In [10]:
model4=load_model("model4.h5")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
y4_pred=model4.predict(X_test_np)
for i in range(len(y4_pred)):
    if(y4_pred[i]<0):
        y4_pred[i]=0
model_4_mse=mean_squared_error(Y_test_np, y4_pred)
model_4_mse

0.0039004348660493216

In [12]:
import pickle

Consider the SVR model with all the features.

In [13]:
model1 = pickle.load(open('model1.pickle', 'rb'))

In [14]:
y1_pred=model1.predict(X_test_np)

In [15]:
for i in range(len(y1_pred)):
    if(y1_pred[i]<0):
        y1_pred[i]=0


In [16]:
model_1_mse=mean_squared_error(Y_test_np, y1_pred)
model_1_mse

0.001602298513105709

Consider the SVR model in which feature reduction with PCA is done.

In [18]:
model2 = pickle.load(open('model2.pickle', 'rb'))

In [19]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
X_pca=pca.fit_transform(X_train_np)
X_test_pca=pca.transform(X_test_np)

In [20]:
y2_pred=model2.predict(X_test_pca)
for i in range(len(y2_pred)):
    if(y2_pred[i]<0):
        y2_pred[i]=0


In [21]:
model_2_mse=mean_squared_error(Y_test_np, y2_pred)
model_2_mse

0.0010448858154379652

From the above evaluation, we can conclude that SVR model using feature reduction gives best result in the 4 models considered here.